

Build a deep learning model to classify the mnist digits dataset with Batch Normalization.
Build a Feed Forward Neural Network for any problems with keras tuner.





In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [3]:
# Define a function to build the model with a specific optimizer
def build_model(optimizer):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [4]:
# List of optimizers to try
optimizers = ['sgd', 'adam', 'rmsprop']

for optimizer_name in optimizers:
    # Build the model with the current optimizer
    model = build_model(optimizer_name)
    
    print(f"\nTraining with {optimizer_name.upper()} optimizer:")
    
    # Train the model
    model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))
    
    # Evaluate the model
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print(f'Test accuracy with {optimizer_name.upper()} optimizer: {test_acc}')



Training with SGD optimizer:
Epoch 1/5
938/938 [==============================] - 49s 51ms/step - loss: 0.1997 - accuracy: 0.9477 - val_loss: 0.0813 - val_accuracy: 0.9791
Epoch 2/5
938/938 [==============================] - 52s 56ms/step - loss: 0.0739 - accuracy: 0.9814 - val_loss: 0.0574 - val_accuracy: 0.9844
Epoch 3/5
938/938 [==============================] - 73s 78ms/step - loss: 0.0530 - accuracy: 0.9867 - val_loss: 0.0465 - val_accuracy: 0.9869
Epoch 4/5
938/938 [==============================] - 71s 75ms/step - loss: 0.0422 - accuracy: 0.9895 - val_loss: 0.0414 - val_accuracy: 0.9881
Epoch 5/5
313/313 [==============================] - 2s 7ms/step - loss: 0.0379 - accuracy: 0.9885
Test accuracy with SGD optimizer: 0.9884999990463257

Training with ADAM optimizer:
Epoch 1/5
938/938 [==============================] - 71s 74ms/step - loss: 0.1062 - accuracy: 0.9692 - val_loss: 0.0593 - val_accuracy: 0.9815
Epoch 2/5
938/938 [==============================] - 61s 65ms/step - los

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import RandomSearch

C:\Users\Dell\AppData\Local\Temp\ipykernel_20648\3405580802.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [6]:
# Load and preprocess the dataset (example with MNIST for simplicity)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values to be between 0 and 1

In [7]:
# Define the model building function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Tune the number of layers and units in each layer
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                               activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate and optimizer
    optimizer_choice = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    if optimizer_choice == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [8]:

# Instantiate the RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    directory='tuner_results',
    project_name='binary_classification'
)


Reloading Tuner from tuner_results\binary_classification\tuner0.json


In [9]:

# Perform the hyperparameter search
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
final_model = tuner.hypermodel.build(best_hps)


In [10]:
# Train the final model
final_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the final model
test_loss, test_acc = final_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/10
1875/1875 [==============================] - 13s 6ms/step - loss: -4861509120.0000 - accuracy: 0.1123 - val_loss: -23186921472.0000 - val_accuracy: 0.1135
Epoch 2/10
1875/1875 [==============================] - 11s 6ms/step - loss: -104669618176.0000 - accuracy: 0.1124 - val_loss: -242031853568.0000 - val_accuracy: 0.1135
Epoch 3/10
1875/1875 [==============================] - 11s 6ms/step - loss: -506690502656.0000 - accuracy: 0.1124 - val_loss: -879890530304.0000 - val_accuracy: 0.1135
Epoch 4/10
1875/1875 [==============================] - 10s 6ms/step - loss: -1429346451456.0000 - accuracy: 0.1124 - val_loss: -2166843637760.0000 - val_accuracy: 0.1135
Epoch 5/10
1875/1875 [==============================] - 10s 5ms/step - loss: -3115012587520.0000 - accuracy: 0.1124 - val_loss: -4364715425792.0000 - val_accuracy: 0.1135
Epoch 6/10
1875/1875 [==============================] - 11s 6ms/step - loss: -5841568661504.0000 - accuracy: 0.1124 - val_loss: -7770374930432.0000 - val_